## Extreme Learning Machine

In [5]:
import time
import numpy as np

def elm_fit(X, target, h, W=None):
    start_time = time.time()
    if W is None:
        W = np.random.uniform(-.1, .1, (h, len(X[0])))
    
    Hinit = X @ W.T
    H = 1 / (1 + np.exp(-Hinit))
    Ht = H.T
    Hp = np.linalg.inv(Ht @ H) @ Ht
    beta = Hp @ target
    y = H @ beta
    mape = sum(abs(y - target) / target) * 100 / len(target)
    
    execution = time.time() - start_time
    print("Waktu eksekusi: %s detik" %execution)
    
    return W, beta, mape

In [6]:
def elm_predict(X, W, b, round_output=False):
    Hinit = X @ W.T
    H = 1 / (1 + np.exp(-Hinit))
    y = H @ b

    if round_output:
        y = [int(round(x)) for x in y]
    
    return y

In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = iris.target
Y += 1

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)
W, b, mape = elm_fit(X_train, y_train, 3)

print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)

print('Output:', output)
print('True :', y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.0 detik
MAPE: 13.08493604680845
Output: [3, 3, 2, 1, 1, 1, 2, 3, 2, 1, 2, 1, 2, 2, 3, 3, 1, 3, 3, 2, 2, 2, 2, 1, 2, 1, 2, 2, 3, 2, 1, 2, 3, 1, 3, 2, 3, 1, 1, 3, 1, 2, 3, 1, 2]
True : [3 2 2 1 1 1 2 3 2 1 3 1 2 2 3 3 1 3 3 2 2 2 3 1 2 1 2 2 3 2 1 2 3 1 3 2 3
 1 1 2 1 2 3 1 2]
Accuracy: 0.9111111111111111


In [9]:
W, b, mape = elm_fit(X_train, y_train, 5)
print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.0009965896606445312 detik
MAPE: 8.58349198433284
Accuracy: 1.0


In [11]:
W, b, mape = elm_fit(X_train, y_train, 7)
print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.0009992122650146484 detik
MAPE: 8.285936077000871
Accuracy: 1.0


In [12]:
W, b, mape = elm_fit(X_train, y_train, 10)
print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.0 detik
MAPE: 7.729482913123249
Accuracy: 0.9777777777777777


In [13]:
W, b, mape = elm_fit(X_train, y_train, 30)
print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)
print('Accuracy:', accuracy)

Waktu eksekusi: 0.019013643264770508 detik
MAPE: 149.31093882959678
Accuracy: 0.0


## Backpropagation

In [14]:
def sig(x):
    return 1 / (1 + np.exp(-x))

def sigd(x):
    return sig(x) * (1 - sig(x))

def onehot_enc(lbl, min_val=0):
    mi = min(lbl)
    enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
    for i, x in enumerate(lbl):
      enc[i, x - mi] = 1
    return enc

def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]

In [30]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    start_time = time.time()
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j+1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i+1]) for i in range(len(layer_conf) - 1)], dtype=object)
    dw = np.array([np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)], dtype=object)
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1] = 1

    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0
        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])

            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

            for L in range(len(layer_conf) - 1, 1, -1):
                din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))

            w += dw
        mse /= len(X)

        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    execution = time.time()- start_time
    # print("Waktu eksekusi: %s detik" % execution)

    return w, epoch, mse

In [31]:
def bp_predict(X, w):
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []
    n.append(np.empty(len(w[-1][0])))
    for x in X:
      n[0][:-1] = x
      for L in range(0, len(w)):
        nin[L] = np.dot(n[L], w[L])
        n[L + 1][:len(nin[L])] = sig(nin[L])
      predict.append(n[-1].copy())
    return predict

In [32]:
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 2),learn_rate=.1, max_epoch=100, max_error=.5, print_per_epoch=25)

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Epochs:', ep)
print('MSE:', mse)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 3.4328209305117845
Epoch 50, MSE: 3.4306377758556503
Epoch 75, MSE: 3.429932320325312
Epoch 100, MSE: 3.4295846961048833
Epochs: 100
MSE: 3.4295846961048833
Output: [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
True : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy: 0.5777777777777777
